## Data PreProcessing

* This notebook preprocesses YouTube video transcripts for the multimodal AI chatbot project. 
* It loads metadata and transcripts, cleans and lemmatizes text, splits into chunks using LangChain, and saves processed data for vector database integration.

In [1]:
%pip install langchain spacy nltk python-dotenv --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --upgrade pip --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip uninstall -y pinecone-client --quiet
%pip install pinecone-client==3.0.0 --quiet
%pip install -U langchain --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import re
import logging
import pandas as pd
import nltk

from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize

from langchain_core.documents.base import Document
from langchain_text_splitters.character import RecursiveCharacterTextSplitter

from pinecone import Pinecone, ServerlessSpec

In [5]:
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [6]:
metadata_file = 'data/ServiceNow_Youtube_Metadata_Clean.csv'
transcript_file = 'data/video_metadata_with_transcripts.csv'
output_file = 'data/processed_transcripts.csv'
validation_report = 'docs/validation_report.txt'

for file in [metadata_file, transcript_file]:
    if not os.path.exists(file):
        logging.error(f'File not found: {file}')
        raise FileNotFoundError(f'File not found: {file}')

In [7]:
os.makedirs('data', exist_ok=True)
os.makedirs('docs', exist_ok=True)

In [8]:
df_meta = pd.read_csv('data/ServiceNow_Youtube_Metadata_Clean.csv')
print("Metadata columns:", df_meta.columns.tolist())

df_trans = pd.read_csv('data/video_metadata_with_transcripts.csv')
print("Transcript columns:", df_trans.columns.tolist())

Metadata columns: ['Number', 'Youtube_link', 'Subject', 'title', 'channel', 'description', 'length', 'publish_date', 'views', 'error']
Transcript columns: ['Number', 'Youtube_link', 'Subject', 'title', 'channel', 'description', 'length', 'publish_date', 'views', 'error', 'transcript']


## Data Preprocessing
* Lemmatization & Tokenization
* Normalizing the columns
* Splitting the Chunks


In [9]:
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    if pd.isna(text):
        logging.warning('Empty text encountered in clean_text')
        return ''
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text.lower().strip()

def tokenize_and_lemmatize(text):
    if not text:
        logging.warning('Empty text encountered in tokenize_and_lemmatize')
        return ''
    tokens = word_tokenize(text)
    lemmatized = [lemmatizer.lemmatize(token) for token in tokens if token]
    return ' '.join(lemmatized)

def load_and_process_transcripts(transcript_path):
    df = pd.read_csv(transcript_path)
    
    # Normalize columns (make sure names are lowercase and consistent)
    df = df.rename(columns={
        'Youtube_link': 'youtube_link',
        'Number': 'video_id',
        'Subject': 'subject',
        'title': 'title',
        'transcript': 'transcript'
    })
    
    # Clean and lemmatize transcript text
    df['cleaned_transcript'] = df['transcript'].apply(clean_text).apply(tokenize_and_lemmatize)
    
    logging.info(f'Loaded and processed {len(df)} transcript records')
    return df

def prepare_langchain_docs(df):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = []
    missing_transcripts = []
    
    for _, row in df.iterrows():
        if not row['cleaned_transcript']:
            missing_transcripts.append(row['video_id'])
            logging.warning(f'Missing transcript for video_id {row["video_id"]}')
            continue
        
        metadata = {
            'title': row['title'],
            'url': row['youtube_link'],
            'subject': row['subject'],
            'video_id': row['video_id'],
        }
        
        chunks = text_splitter.split_text(row['cleaned_transcript'])
        for i, chunk in enumerate(chunks):
            doc = Document(page_content=chunk, metadata={**metadata, 'chunk_id': f"{row['video_id']}_{i}"})
            docs.append(doc)
            
        logging.info(f'Created {len(chunks)} chunks for video_id {row["video_id"]}')
        
    logging.info(f'Created {len(docs)} total LangChain documents')
    return docs, missing_transcripts

# Usage example:
df_clean = load_and_process_transcripts(transcript_file)
langchain_docs, missing = prepare_langchain_docs(df_clean)

print(f"Processed {len(langchain_docs)} document chunks.")
if missing:
    print(f"Missing transcripts for video IDs: {missing}")


2025-06-26 20:46:30,137 - INFO - Loaded and processed 22 transcript records
2025-06-26 20:46:30,141 - INFO - Created 14 chunks for video_id 1
2025-06-26 20:46:30,147 - INFO - Created 21 chunks for video_id 2
2025-06-26 20:46:30,150 - INFO - Created 17 chunks for video_id 3
2025-06-26 20:46:30,154 - INFO - Created 17 chunks for video_id 4
2025-06-26 20:46:30,157 - INFO - Created 11 chunks for video_id 5
2025-06-26 20:46:30,164 - INFO - Created 14 chunks for video_id 6
2025-06-26 20:46:30,169 - INFO - Created 17 chunks for video_id 7
2025-06-26 20:46:30,172 - INFO - Created 8 chunks for video_id 8
2025-06-26 20:46:30,181 - INFO - Created 15 chunks for video_id 9
2025-06-26 20:46:30,186 - INFO - Created 21 chunks for video_id 10
2025-06-26 20:46:30,193 - INFO - Created 22 chunks for video_id 11
2025-06-26 20:46:30,195 - INFO - Created 5 chunks for video_id 12
2025-06-26 20:46:30,200 - INFO - Created 14 chunks for video_id 13
2025-06-26 20:46:30,204 - INFO - Created 21 chunks for video_id 

Processed 328 document chunks.


## Preprocess Transcripts
* Loading and Preprocessing transcripts
* Converting to LangChain docs and Split Chunks
* Saving Processed Data as processed_transcripts.csv file
* Validating creating Data validation file as validation_report.txt


In [10]:
def preprocess_transcripts():
    df_clean = load_and_process_transcripts(transcript_file)
    langchain_docs, missing_transcripts = prepare_langchain_docs(df_clean)
    processed_data = []
    for doc in langchain_docs:
        processed_data.append({
            'video_id': doc.metadata['video_id'],    
            'chunk_id': doc.metadata['chunk_id'],
            'text': doc.page_content,
            'subject': doc.metadata['subject']
        })
    df_processed = pd.DataFrame(processed_data)
    if df_processed.empty:
        logging.error('No processed data generated')
        raise ValueError('No processed data generated')
    try:
        df_processed.to_csv(output_file, index=False, encoding='utf-8')
        logging.info(f'Saved {len(df_processed)} transcript chunks to {output_file}')
    except Exception as e:
        logging.error(f'Error saving processed data: {e}')
        raise
    df_transcripts = pd.read_csv(transcript_file)
    metadata_success = df_transcripts['title'].notnull().sum()
    missing_trans = df_transcripts['transcript'].isna().sum()
    total_chunks = len(df_processed)
    sample_chunk = df_processed['text'].iloc[0] if not df_processed.empty else 'No chunks'
    with open(validation_report, 'w') as f:
        f.write(f'Transcripts: {len(df_transcripts)} videos, {missing_trans} missing\n')
        f.write(f'Processed Chunks: {total_chunks}\n')
        f.write(f'Sample Chunk:\n{sample_chunk}\n')
        if missing_transcripts:
            f.write(f'Missing transcripts for video IDs: {missing_transcripts}\n')
    logging.info(f'Validation report saved to {validation_report}')
    print(f'Metadata success: {metadata_success}/{len(df_transcripts)}')
    print(f'Missing transcripts: {missing_trans}/{len(df_transcripts)}')
    print(f'Processed chunks: {total_chunks}')
    if missing_transcripts:
        print(f'Missing transcripts for video IDs: {missing_transcripts}')
    print(df_processed.head())

preprocess_transcripts()


2025-06-26 20:46:30,564 - INFO - Loaded and processed 22 transcript records
2025-06-26 20:46:30,568 - INFO - Created 14 chunks for video_id 1
2025-06-26 20:46:30,571 - INFO - Created 21 chunks for video_id 2
2025-06-26 20:46:30,577 - INFO - Created 17 chunks for video_id 3
2025-06-26 20:46:30,585 - INFO - Created 17 chunks for video_id 4
2025-06-26 20:46:30,589 - INFO - Created 11 chunks for video_id 5
2025-06-26 20:46:30,596 - INFO - Created 14 chunks for video_id 6
2025-06-26 20:46:30,602 - INFO - Created 17 chunks for video_id 7
2025-06-26 20:46:30,603 - INFO - Created 8 chunks for video_id 8
2025-06-26 20:46:30,608 - INFO - Created 15 chunks for video_id 9
2025-06-26 20:46:30,615 - INFO - Created 21 chunks for video_id 10
2025-06-26 20:46:30,620 - INFO - Created 22 chunks for video_id 11
2025-06-26 20:46:30,622 - INFO - Created 5 chunks for video_id 12
2025-06-26 20:46:30,628 - INFO - Created 14 chunks for video_id 13
2025-06-26 20:46:30,632 - INFO - Created 21 chunks for video_id 

Metadata success: 21/22
Missing transcripts: 0/22
Processed chunks: 328
   video_id chunk_id                                               text  \
0         1      1_0  hey folk how you doing chris thanky here and i...   
1         1      1_1  and issue and in those project youre going to ...   
2         1      1_2  project the project status report any issue th...   
3         1      1_3  be it would be crazy right it would take you f...   
4         1      1_4  person and asking him this question and asking...   

                                             subject  
0  An AI Agent that knows everything about your P...  
1  An AI Agent that knows everything about your P...  
2  An AI Agent that knows everything about your P...  
3  An AI Agent that knows everything about your P...  
4  An AI Agent that knows everything about your P...  


In [11]:
# Verify processed data
try:
    df_processed = pd.read_csv('data/processed_transcripts.csv')
    print(f'Total chunks: {len(df_processed)}')
    print(f'Unique videos: {df_processed["video_id"].nunique()}')
    print(df_processed[['video_id', 'chunk_id', 'text', 'subject']].head())
except FileNotFoundError:
    logging.error(f'Output file not found: {output_file}')
    print(f'Output file not found: {output_file}')

Total chunks: 328
Unique videos: 22
   video_id chunk_id                                               text  \
0         1      1_0  hey folk how you doing chris thanky here and i...   
1         1      1_1  and issue and in those project youre going to ...   
2         1      1_2  project the project status report any issue th...   
3         1      1_3  be it would be crazy right it would take you f...   
4         1      1_4  person and asking him this question and asking...   

                                             subject  
0  An AI Agent that knows everything about your P...  
1  An AI Agent that knows everything about your P...  
2  An AI Agent that knows everything about your P...  
3  An AI Agent that knows everything about your P...  
4  An AI Agent that knows everything about your P...  


In [12]:
df = pd.read_csv("Data/processed_transcripts.csv")
def clean_text(text):
    text = re.sub(r"\s+", " ", text)  # normalize whitespace

    # List of known filler phrases
    filler_patterns = [
        r"\b(hi|hey|hello|folks|how you doing|thank you|thanks|welcome|today|i am|i'm|this is .*?)\b",
        r"\b(thanky here|and i\b|let's talk about|so today|in this video)\b",
        r"\b(folk|id like|to you|to this session|chris)\b",
    ]

    for pattern in filler_patterns:
        text = re.sub(pattern, "", text, flags=re.IGNORECASE)

    text = re.sub(r"\s{2,}", " ", text)  # clean double spaces
    return text.strip()
def chunk_transcript_by_sentences(text, max_sentences=4):
    sentences = sent_tokenize(text)
    return [' '.join(sentences[i:i+max_sentences]) for i in range(0, len(sentences), max_sentences)]
all_chunks = []

for transcript in df["text"]:  # Replace 'text' if your column is named differently
    if pd.isna(transcript):
        continue
    cleaned = clean_text(transcript)
    chunks = chunk_transcript_by_sentences(cleaned)
    all_chunks.extend(chunks)

# Save cleaned chunks for use in RAG
output_df = pd.DataFrame({
    "chunk_id": [f"chunk_{i}" for i in range(len(all_chunks))],
    "video_id": df["video_id"].iloc[0] if not df.empty else "unknown",  # Assuming all chunks are from the same video
    "text": all_chunks
})

output_df.to_csv("Data/processed_cleaned_chunks.csv", index=False)
output_df.head()

,chunk_id,video_id,text
0,chunk_0,1,and to you where i plan to show you the greate...
1,chunk_1,1,and issue and in those project youre going to ...
2,chunk_2,1,project the project status report any issue th...
3,chunk_3,1,be it would be crazy right it would take you f...
4,chunk_4,1,person and asking him this question and asking...


In [14]:
def store_in_pinecone(docs, index_name='youtube-transcripts', preview_file='chunk_previews.txt'):
    load_dotenv()
    pinecone_api_key = os.getenv('PINECONE_API_KEY')
    if not pinecone_api_key:
        logging.error('PINECONE_API_KEY not found in .env')
        raise ValueError('PINECONE_API_KEY not found')

    # Initialize Pinecone client
    pc = Pinecone(api_key=pinecone_api_key)

    # Create index if not exists
    if index_name not in [idx['name'] for idx in pc.list_indexes()]:
        pc.create_index(
            name=index_name,
            dimension=384,
            metric='cosine',
            spec=ServerlessSpec(cloud='aws', region='us-east-1')
        )

    embedder = SentenceTransformer('all-MiniLM-L6-v2')
    index = pc.Index(index_name)
    vectors = []

    with open(preview_file, 'w', encoding='utf-8') as f:
        for i, doc in enumerate(docs):
            embedding = embedder.encode(doc.page_content, show_progress_bar=False).tolist()
            # Clean metadata: replace NaN/None with empty string
            clean_metadata = {k: ("" if pd.isna(v) or v is None else v) for k, v in doc.metadata.items()}
            clean_metadata['text'] = doc.page_content
            vectors.append({
                'id': doc.metadata['chunk_id'],
                'values': embedding,
                'metadata': clean_metadata
            })

            # Write a short preview to the file
            preview_text = doc.page_content[:100].replace('\n', ' ')  # limit to 100 chars, no new lines
            f.write(f"[{i+1}] Chunk ID: {doc.metadata['chunk_id']} | Video ID: {doc.metadata['video_id']} | Text: {preview_text}...\n")

            if len(vectors) >= 100:
                index.upsert(vectors=vectors)
                vectors = []

        if vectors:
            index.upsert(vectors=vectors)

    #logging.info(f'Upserted {len(docs)} vectors to Pinecone index {index_name}')
    #logging.info(f'Chunk previews saved to {preview_file}')
df = pd.read_csv("Data/processed_cleaned_chunks.csv")

# Convert cleaned chunks to LangChain Document format
langchain_docs = [
    Document(
        page_content=row["text"],
        metadata={
            "chunk_id": row["chunk_id"],
            "video_id": row["video_id"]
        }
    )
    for _, row in df.iterrows()
]

store_in_pinecone(langchain_docs, index_name='youtube-transcripts', preview_file='docs/chunk_previews.txt') 


2025-06-26 20:49:09,086 - INFO - Use pytorch device_name: cpu
2025-06-26 20:49:09,087 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [ ]:
def validate_data(df, df_processed):
    """Validate input and processed data."""
    # Check for duplicates
    duplicates = df[df.duplicated(subset=['video_id', 'youtube_link'], keep=False)]
    if not duplicates.empty:
        logging.warning(f'Found {len(duplicates)} duplicate video_id/youtube_link entries')
    
    # Validate YouTube links
    invalid_links = df[~df['youtube_link'].str.contains(r'youtube\.com|youtu\.be', na=False)]
    if not invalid_links.empty:
        logging.warning(f'Found {len(invalid_links)} invalid YouTube links')
    
    # Chunk length stats
    chunk_lengths = df_processed['text'].str.len()
    chunk_stats = {
        'avg_length': chunk_lengths.mean(),
        'min_length': chunk_lengths.min(),
        'max_length': chunk_lengths.max()
    }
    logging.info(f'Chunk length stats: {chunk_stats}')
    return chunk_stats

# In preprocess_transcripts(), before saving:
chunk_stats = validate_data(df_clean, df_processed)
with open(validation_report, 'a') as f:
    f.write(f'Chunk Length Stats: Average={chunk_stats["avg_length"]:.1f}, Min={chunk_stats["min_length"]}, Max={chunk_stats["max_length"]}\n')

In [ ]:
# Update project documentation
with open('docs/project_log.md', 'a') as f:
    f.write('## Data Collection and Preprocessing\n')
    f.write('- Loaded YouTube video from `data/SNOW_YT_Videos.csv`.\n')
    f.write('- Loaded YouTube video metadata to `data/ServiceNow_Youtube_Metadata_Clean.csv`.\n')
    f.write('- Loaded YouTube video transcripts to `data/video_metadata_with_transcripts.csv`.\n')
    f.write('- Preprocessed transcripts with NLTK lemmatization and LangChain text splitting (chunk_size=500, overlap=50).\n')
    f.write(f'- Processed 22 videos, generating 328 chunks.\n')
    f.write('- Saved processed data to `data/processed_transcripts.csv`.\n')
    f.write('- Challenges: Resolved KeyError by standardizing column names (e.g., Number to video_id).\n')
    f.write('- Validation report saved to `docs/validation_report.txt`.\n')
    f.write('- Chunk Preview Data saved to `docs/chunk_preview.csv`.\n')
    f.write(f'  Average: {chunk_stats["avg_length"]:.1f}\n')
    f.write(f'  Minimum: {chunk_stats["min_length"]}\n')
    f.write(f'  Maximum: {chunk_stats["max_length"]}\n')